In [1]:
# this notebook requires nuztf installation
%load_ext autoreload
%autoreload 2

In [1]:
from nuztf.skymap_scanner import SkymapScanner

import pandas as pd
import numpy as np

import os

from tqdm.notebook import tqdm

In [9]:
candidate_filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-healpix/test2/compareKasliwal_confirmed.ods"
#candidate_filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-healpix/test2/S190426c_ampel_kasliwal.ods"
#candidate_filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-healpix/test2/S200115j_rev_0/TransientTable.csv"


O3_candidates = pd.read_excel(candidate_filename)
#data_df = pd.read_csv(candidate_filename, sep=";")

#print(data_df.head())

O3_candidates = O3_candidates.sort_values(by="Kilonovaness", ascending=False)

gw_map = O3_candidates["Healpix map"].unique()
gw_names = [gw[:gw.find(".")] for gw in gw_map]
print(len(O3_candidates))

80


In [10]:
file_dir = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-healpix/test2/"

plot_dir = os.path.join(file_dir, "visualisePlots")

try:
    os.makedirs(plot_dir)
except:
    pass

table_name = "TransientTable.csv"
os.listdir(file_dir)

transient_names = []

ampel_candidates = pd.DataFrame()

for transient in os.listdir(file_dir):
    tableFile_dir = os.path.join(file_dir, transient, table_name)
    if os.path.isfile(tableFile_dir):
        table_df = pd.read_csv(tableFile_dir, sep=";")
        transient_names.append(table_df["Healpix map"].iloc[0])

        #pd.read_csv(tableFile_dir, sep=";")

        ampel_candidates = pd.concat([ampel_candidates, table_df])

print(len(ampel_candidates))

try:
    possibleSelectKeys = ampel_candidates.keys()
except:
    print("No transient tables found.")

transient_names.extend(["ALL", "FOLLOWUP"])
print(transient_names)

440
['S190426c.fits.gz,1', 'S190901ap.fits.gz,0', 'S190910d.fits.gz,0', 'S190910h.fits.gz,0', 'S190930t.fits.gz,0', 'S191213g.fits.gz', 'S200115j.fit.gz,0', 'ALL', 'FOLLOWUP']


50

In [2]:
def print_nuztf_summary(gw_name, ztf_ids, data_subset):
    prob_threshold = 0.9
    window = 3

    #ztf_ids = ["ZTF19aatubsj"]

    #print(ztf_ids)

    scanner = SkymapScanner(
        event=gw_name,
        prob_threshold=prob_threshold,
        n_days=window,
    )
    scanner.add_to_cache_by_names(ztf_ids=ztf_ids, with_history=False)

    print(scanner.cache.keys())

    #alert["kilonova_eval"]["kilonovaness"]

    for ztf_name, cand_dict in scanner.cache.items():
        if data_subset.get("Kilonovaness") is not None:
            #print(ztf_name, data_subset[data_subset["ztf_id"] == ztf_name]["Kilonovaness"].values)
            #print(ztf_name, data_subset[data_subset["ztf_id"] == ztf_name]["Kilonovaness"].values[0])
            kilonovaness = data_subset[data_subset["ztf_id"] == ztf_name]["Kilonovaness"].values[0]
        else:
            kilonovaness = None
        #print(ztf_name, data_subset[data_subset["ztf_id"] == ztf_name]["Kilonovaness"].values[0])

        if data_subset.get("Ampel z") is not None:
            z = data_subset[data_subset["ztf_id"] == ztf_name]["Ampel z"].values[0]
        else:
            z = None

        cand_dict.update({"kilonova_eval": {"kilonovaness": kilonovaness, "z": z}})

    print(scanner.cache.keys())

    scanner.create_candidate_summary()

In [14]:
ampel_and_o3_candidates = ampel_candidates[ampel_candidates["ztf_id"].isin(O3_candidates["ztf_id"])]
gw_map = ampel_and_o3_candidates["Healpix map"].unique()
gw_names = [gw[:gw.find(".")] for gw in gw_map]

print(gw_names)


for i in tqdm(range(0, len(gw_names))):
    gw_name = gw_names[i]
    data_subset = (ampel_and_o3_candidates[ampel_and_o3_candidates["Healpix map"] == gw_map[i]]).sort_values(by="Kilonovaness", ascending=False)

    ztf_ids = np.char.strip(data_subset["ztf_id"].to_numpy(dtype="str"))

    print(gw_name, ztf_ids)
    print_nuztf_summary(gw_name, ztf_ids, data_subset)

['S190426c', 'S190901ap', 'S190910d', 'S190910h', 'S190930t', 'S191213g']


  0%|          | 0/6 [00:00<?, ?it/s]

S190426c ['ZTF19aaskmit' 'ZTF19aaslpds' 'ZTF19aaslfre' 'ZTF19aaslzjf'
 'ZTF19aasknld' 'ZTF19aaslswv' 'ZTF19aaslozu' 'ZTF19aatazmc']


  0%|          | 0/3145728 [00:00<?, ?it/s]

  0%|          | 0/1350 [00:00<?, ?it/s]

...

  0%|          | 0/3145728 [00:00<?, ?it/s]

  0%|          | 0/5336 [00:00<?, ?it/s]

dict_keys(['ZTF19acyfoha', 'ZTF19acyldun', 'ZTF19acymaru', 'ZTF19acymlhi', 'ZTF19acyldum'])
dict_keys(['ZTF19acyfoha', 'ZTF19acyldun', 'ZTF19acymaru', 'ZTF19acymlhi', 'ZTF19acyldum'])


  0%|          | 0/5 [00:00<?, ?it/s]

0.04138
0.05727
0.175709262
0.142
None


In [19]:
ampel_not_o3_candidates = ampel_candidates[np.logical_not(ampel_candidates["ztf_id"].isin(O3_candidates["ztf_id"]))]
ampel_not_o3_candidates = ampel_not_o3_candidates[ampel_not_o3_candidates["Kilonovaness"] >= 20]

len(ampel_not_o3_candidates)

27

In [20]:
ampel_not_o3_candidates = ampel_candidates[np.logical_not(ampel_candidates["ztf_id"].isin(O3_candidates["ztf_id"]))]
gw_map = ampel_and_o3_candidates["Healpix map"].unique()
gw_names = [gw[:gw.find(".")] for gw in gw_map]

print(gw_names)


for i in tqdm(range(0, len(gw_names))):
    gw_name = gw_names[i]
    data_subset = (ampel_and_o3_candidates[ampel_and_o3_candidates["Healpix map"] == gw_map[i]]).sort_values(by="Kilonovaness", ascending=False)

    ztf_ids = np.char.strip(data_subset["ztf_id"].to_numpy(dtype="str"))

    print(gw_name, ztf_ids)
    print_nuztf_summary(gw_name, ztf_ids, data_subset)

['S190426c', 'S190901ap', 'S190910d', 'S190910h', 'S190930t', 'S191213g']


  0%|          | 0/6 [00:00<?, ?it/s]

S190426c ['ZTF19aaskmit' 'ZTF19aaslpds' 'ZTF19aaslfre' 'ZTF19aaslzjf'
 'ZTF19aasknld' 'ZTF19aaslswv' 'ZTF19aaslozu' 'ZTF19aatazmc']


  0%|          | 0/3145728 [00:00<?, ?it/s]

  0%|          | 0/1350 [00:00<?, ?it/s]

...

  0%|          | 0/3145728 [00:00<?, ?it/s]

  0%|          | 0/5336 [00:00<?, ?it/s]

dict_keys(['ZTF19acyfoha', 'ZTF19acyldun', 'ZTF19acymaru', 'ZTF19acymlhi', 'ZTF19acyldum'])
dict_keys(['ZTF19acyfoha', 'ZTF19acyldun', 'ZTF19acymaru', 'ZTF19acymlhi', 'ZTF19acyldum'])


  0%|          | 0/5 [00:00<?, ?it/s]

0.04138
0.05727
0.175709262
0.142
None


In [27]:
for i in tqdm(range(0, len(gw_names)- 0)):
    #print(i)
    gw_name = gw_names[i]
    data_subset = (O3_candidates[O3_candidates["Healpix map"] == gw_map[i]]).sort_values(by="Kilonovaness", ascending=False)

    #print(data_subset.head(20))
    
    ztf_ids = np.char.strip(data_subset["ztf_id"].to_numpy(dtype="str"))

    #print(data_subset.head())

    print(gw_name, ztf_ids)
    print_nuztf_summary(gw_name, ztf_ids, data_subset)

    # plan: nimm aus data_df gw_name, 
    # nur für candidates mit dem gw_name, mache pdfs
    # evtl versuche kilonovaness in pdf zu integrieren?

  0%|          | 0/1 [00:00<?, ?it/s]

S190426c ['ZTF19aasknld' 'ZTF19aaslpds' 'ZTF19aaskmit' 'ZTF19aaslfre'
 'ZTF19aaslswv' 'ZTF19aaslzjf' 'ZTF19aatazmc' 'ZTF19aaslolf'
 'ZTF19aaslozu' 'ZTF19aaslphi' 'ZTF19aaslszp' 'ZTF19aaslvwn'
 'ZTF19aaslzfk' 'ZTF19aasmddt' 'ZTF19aasmdir' 'ZTF19aasmekb'
 'ZTF19aasmftm' 'ZTF19aasmzqf' 'ZTF19aassfws' 'ZTF19aasshpf']


  0%|          | 0/3145728 [00:00<?, ?it/s]

  0%|          | 0/1350 [00:00<?, ?it/s]

dict_keys(['ZTF19aasknld', 'ZTF19aaslpds', 'ZTF19aaskmit', 'ZTF19aaslfre', 'ZTF19aaslswv', 'ZTF19aaslzjf', 'ZTF19aatazmc', 'ZTF19aaslolf', 'ZTF19aaslozu', 'ZTF19aaslphi', 'ZTF19aaslszp', 'ZTF19aaslvwn', 'ZTF19aaslzfk', 'ZTF19aasmddt', 'ZTF19aasmdir', 'ZTF19aasmekb', 'ZTF19aasmftm', 'ZTF19aasmzqf', 'ZTF19aassfws', 'ZTF19aasshpf'])
dict_keys(['ZTF19aasknld', 'ZTF19aaslpds', 'ZTF19aaskmit', 'ZTF19aaslfre', 'ZTF19aaslswv', 'ZTF19aaslzjf', 'ZTF19aatazmc', 'ZTF19aaslolf', 'ZTF19aaslozu', 'ZTF19aaslphi', 'ZTF19aaslszp', 'ZTF19aaslvwn', 'ZTF19aaslzfk', 'ZTF19aasmddt', 'ZTF19aasmdir', 'ZTF19aasmekb', 'ZTF19aasmftm', 'ZTF19aasmzqf', 'ZTF19aassfws', 'ZTF19aasshpf'])


  0%|          | 0/20 [00:00<?, ?it/s]

0.129453341
0.106
0.087
0.090271004
0.102
0.0640993333333333
0.07781
None
None
None
None
None
None
None
None
None
None
None
None
None


In [3]:
candidate_filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-healpix/test2/S190426c_rev_1/TransientTable.csv"

data = pd.read_csv(candidate_filename, sep=";")
candidates = data.sort_values(by="Kilonovaness", ascending=False)

gw_map = candidates["Healpix map"].unique()
gw_names = [gw[:gw.find(".")] for gw in gw_map]
print(len(candidates))

ztf_ids = np.char.strip(data["ztf_id"].to_numpy(dtype="str"))

print(ztf_ids)

print_nuztf_summary(gw_names[0], ztf_ids, data)

42
['ZTF19aasknld' 'ZTF19aaslswv' 'ZTF19aaskmit' 'ZTF19aataxum'
 'ZTF19aaslfre' 'ZTF19aatazmc' 'ZTF19aaslzjf' 'ZTF19aaslpsk'
 'ZTF19aaskikg' 'ZTF19aaskktr' 'ZTF19aaskjsw' 'ZTF19aaslxmg'
 'ZTF19aaskjwt' 'ZTF19aaslpqg' 'ZTF19aaskpbp' 'ZTF19aaskjyj'
 'ZTF19aaskyxh' 'ZTF19aasknfi' 'ZTF19aaslixg' 'ZTF19aaskmpt'
 'ZTF19aaskijn' 'ZTF19aasledp' 'ZTF19aaskiht' 'ZTF19aaskipx'
 'ZTF19aasldad' 'ZTF19aaslrms' 'ZTF19aaslssx' 'ZTF19aaslddj'
 'ZTF19aasldsv' 'ZTF19aaskpux' 'ZTF19aasslob' 'ZTF19aaslfmk'
 'ZTF19aaslovx' 'ZTF19aasrgik' 'ZTF19aasmppj' 'ZTF19aasmqvm'
 'ZTF19aaslldo' 'ZTF19aasmdqa' 'ZTF19aaslell' 'ZTF19aaskihm'
 'ZTF19aaslpds' 'ZTF19aaslozu']


  0%|          | 0/3145728 [00:00<?, ?it/s]

  0%|          | 0/1350 [00:00<?, ?it/s]

dict_keys(['ZTF19aasknld', 'ZTF19aaslswv', 'ZTF19aaskmit', 'ZTF19aataxum', 'ZTF19aaslfre', 'ZTF19aatazmc', 'ZTF19aaslzjf', 'ZTF19aaslpsk', 'ZTF19aaskikg', 'ZTF19aaskktr', 'ZTF19aaskjsw', 'ZTF19aaslxmg', 'ZTF19aaskjwt', 'ZTF19aaslpqg', 'ZTF19aaskpbp', 'ZTF19aaskjyj', 'ZTF19aaskyxh', 'ZTF19aasknfi', 'ZTF19aaslixg', 'ZTF19aaskmpt', 'ZTF19aaskijn', 'ZTF19aasledp', 'ZTF19aaskiht', 'ZTF19aaskipx', 'ZTF19aasldad', 'ZTF19aaslrms', 'ZTF19aaslssx', 'ZTF19aaslddj', 'ZTF19aasldsv', 'ZTF19aaskpux', 'ZTF19aasslob', 'ZTF19aaslfmk', 'ZTF19aaslovx', 'ZTF19aasrgik', 'ZTF19aasmppj', 'ZTF19aasmqvm', 'ZTF19aaslldo', 'ZTF19aasmdqa', 'ZTF19aaslell', 'ZTF19aaskihm', 'ZTF19aaslpds', 'ZTF19aaslozu'])
dict_keys(['ZTF19aasknld', 'ZTF19aaslswv', 'ZTF19aaskmit', 'ZTF19aataxum', 'ZTF19aaslfre', 'ZTF19aatazmc', 'ZTF19aaslzjf', 'ZTF19aaslpsk', 'ZTF19aaskikg', 'ZTF19aaskktr', 'ZTF19aaskjsw', 'ZTF19aaslxmg', 'ZTF19aaskjwt', 'ZTF19aaslpqg', 'ZTF19aaskpbp', 'ZTF19aaskjyj', 'ZTF19aaskyxh', 'ZTF19aasknfi', 'ZTF19aaslixg', '

  0%|          | 0/42 [00:00<?, ?it/s]